# Mini Problem Set: Diagnosis
## Implicates and Implicants for Constraint Programming (50 points)

1. [Introduction](#introduction)
2. [Tree Method](#treemethod)
3. [Prime implicate method](#prime-implicate)
    1. [Detect conflicts](#conflict)
    2. [Test validity](#validity)
    3. [Test unsatisfiability](#unsatisfiable)
    4. [Full algorithm](#full-algorithm)

Make sure you load the dependencies below by highlighting the cell below and pressing Shift + Enter.

In [2]:
%load_ext autoreload
%autoreload 2
from propositional_state_logic import *
from sat_solver import *
from utils import *

## Introduction <a id="introduction"/>
</a>

In this problem set, you'll answer a theoretical question about the tree method and will also implement a prime implicate generator. This generator helps decrease the size of a model to make diagnosis more efficient.

## Question 1: Tree Method (10 Points) <a id="treemethod"/>
</a>

In this lecture we discussed Slagle's Tree Method (1970) which determines prime implicants through a depth-first search of an ordered tree in which the root "holds" all clauses in the theory to be processed. Describe the runtime of the tree method and provide justification.

Your Answer Here

## Prime implicate method <a id="prime-implicate"/></a>

### Please refer to the tutorial entitled "Mini Problem Set Tutorial", for additional information and guidance on the Prime Implicate Method. Tutorial can be found in the same folder as this assignment.


In this HW, we will use the thruster model described in the paper, "Diagnosis: Implicates and Implicants for Constraint Programming" and covered in lecture. We've also used tools from the Pset 3: Diagnosis, and included some additional ones to facilitate the implementation. First, we will define the model and constraints. Next, we will create stand alone methods to identify if a node is valid, if it is unsatisfiable, or if it is satisfiable. Based on these methods, we will write the full algorithm to generate prime implicates, based on the minimal conflicts that we find. 

In [3]:
p = Problem()

# Define the variables for the mini thruster problem with variables: T1, R1, and P3. Returns a Variable object.
# Thrust: T1
T1 = p.add_variable('thruster', type='finite_domain', domain=['thrust', 'nothrust'])
# Thruster: R1
R1 = p.add_variable('runthruster', type='finite_domain', domain=['on', 'off'])
# Pressure before the thruster: P3
P3 = p.add_variable('pressure', type='finite_domain', domain=['high', 'low'])

# Add the theory / problem constraints.
# The thruster only outputs thrust when it is on and when the input from P3 is high
p.add_constraint('runthruster=on & pressure=high => thruster=thrust')
p.add_constraint('runthruster=on & pressure=low => thruster=nothrust')
p.add_constraint('runthruster=off => thruster=nothrust')

# Prints out constraints nicely in LaTeX, so you can check them.
display_constraints(p)

# Define SAT for future use
sat = SATSolver(p)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In the following block of code, we create the structure of the tree over which we will iterate. In a true sceneario, we would not be searching over all of the tree (to save time), but for the purpose of this exercise, we iterate over all the tree to present the different statuses that a node can have: valid, unsatifiable, satisfiable.

In [4]:
thruster_model = {
    frozenset([T1.get_assignment('thrust')]) : {
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('low')]) : {},
        },
    
    frozenset([T1.get_assignment('nothrust')]) : {
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('low')]) : {},
        },
    
    frozenset([R1.get_assignment('on')]) : {
            frozenset([R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([R1.get_assignment('off')]) : {
            frozenset([R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
    
    frozenset([P3.get_assignment('high')]) : {},
    
    frozenset([P3.get_assignment('low')]) : {},
    }

### Detect conflicts <a id="conflict"/>
</a>



One of the pruning rules that we have in our tree is to check for conflicts. Thanks to the helper function `check_consistency` that was provided to us for pset 3, it is easy to program:

In [5]:
def is_conflict(sat, candidate):
    return not sat.check_consistency(candidate)[0]

We check our function using assignments for which we know the result: 

In [6]:
# Example of a conflict: This should return true
# The thruster cannot thrust if the run thruster command is "Off"
is_conflict(sat, frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')]))

True

In [7]:
# Example of an assignment that is not a conflict: This should return false
# This is not a conflict because the thruster will thrust if the thruster command is "On" and the pressure before the thruster is "High"
is_conflict(sat, frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('high')]))

False

### Test validity (10 Points) <a id="validity"/>
</a>

For the second pruning rule, we need to identify if a candidate is valid. A candidate is valid if it is not a conflict and none of its children will be conflicts. We will use a helper function to get all of the children (keys) of a node, based on the structure of the tree we created above.

In [8]:
# Helper Function
def get_all_children(d):
    for key, value in d.items():
        yield key
        if isinstance(value, dict):
            yield from get_all_children(value)


Now that we have this helper function, we can check a candidate's validity.

#### Question 2: Write Code for Validity Here
</a>

In [11]:
# Problem 1: Test Validity

# Internal note: we will delete this when we hand it in to the students

def is_valid(candidate, sat, thruster_model_children):

    # YOUR CODE HERE


The following examples show an example of a valid candidate and a candidate that is not valid:

In [ ]:
# Validity Example 1: Candidate that is Not Valid, Should Return "False"
# Candidate is not valid because it has children such as "T1 = No Thrust, R1 = On, and P3 = High, which are conflicts"
candidate = frozenset([T1.get_assignment('nothrust')])
thruster_model_subset_1 = {
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('low')]) : {},
        }

is_valid(candidate, sat, thruster_model_subset_1)

In [ ]:
# Validity Example 2: Candidate that is Valid, Should Return "True"
# Candidate is valid, because if R1 = off and T1 = No thrust, it does not matter what P3 is. T1 should = No thrust whether P3 is high or low
candidate = frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')])
thruster_model_subset = thruster_model_subset_1[candidate]

is_valid(candidate, sat, thruster_model_subset)

We now run a series of test on candidates to check the "is_valid" function:

In [ ]:
### These will be NBgrader tests
check_validity(is_valid)
test_ok()

### Test Unsatisfiability (10 Points) <a id="unsatisfiable"/>
</a>

We now test for unsatisfiability. Similar to the test for validity, we need to check if a candidate is a conflict, and if all of its children are also conflicts. If any children are consistent with the given constraints (i.e. are not conflicts), it means the candidate is NOT unsatisfiable.

#### Question 3: Write Code for Unsatisfiability Here
</a>

In [27]:
# Problem 2: Test for Unsatisfiability
# NOTE: IF THE CANDIDATE ITSELF CONFLICTS WITH THE THEORY, THE CANDIDATE IS UNSATISFIABLE

# Internal note: we will delete this when we hand it in to the students

def is_unsatisfiable(candidate, sat, thruster_model_children):

    # YOUR CODE HERE


The following examples show an example of an unsatisfiable candidate and a candidate that is not unsatisfiable:

In [ ]:
# Unsatisfiability Example 1: Candidate that is Unsatisfiable, Should return "True"
# Candidate "T1 = Thrust, R1 = Off" is unsatisfiable because the candidate is a conflict when P3 is both "High" and "Low", 
# because T1 cannot thrust when R1 is off
candidate = frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')])
thruster_model_subset_3 = {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            }

is_unsatisfiable(candidate, sat, thruster_model_subset_3)

In [ ]:
# Unsatisfiability Example 2: Candidate that is Not Unsatisfiable, Should return "False"
# This candidate is not unsatisfiable because some of its children are not conflicts. For example,"T1 = Thrust, R1 = On, P3 = High" is not a conflict,
# because it is a consistent assignement. This is because T1 should thrust when R1 = On and P3 = High
candidate = frozenset([T1.get_assignment('thrust')])
thruster_model_subset_4 = {
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('low')]) : {},
        }

is_unsatisfiable(candidate, sat, thruster_model_subset_4)

We now run a series of test on candidates to check the "is_unsatisfiable" function:

In [ ]:
### These will be NBgrader tests
check_unsatisfiability(is_unsatisfiable)
test_ok()

### Full algorithm: Minimal Conflict Generator (20 Points) <a id="full-algorithm"/>
</a>

Finally, you will iterate through each of the candidates received from the candidate generator, and determine if they should be either sent back to the generator, converted to an implicate, or added to the solutions list. Your `is_valid` and `is_unsatisfiable` functions from Questions 1 & 2 will be helpful here.

Algorithm reminder: if a candidate is valid, it should be sent back to the generator. If a candidate is unsatisfiable, it should be added to the solutions list. If a candidate is satisfiable, it should be converted to an implicate. In this example, assume that a candidate is satisfiable if it is not valid and is not unsatisfiable.

In [33]:
def prime_implicate_finder(sat, candidateList):
    ## sat represents the theory behind the represented model
    ## CandidateList: List of Tuples (candidate, thruster_model_children), 
        # where the candidate is the item to be tested and 
        # thruster_model_children is a dictionary holding the children of the given candidate
    ## generatorAdditions: add candidate to generatorAdditions list when it should go to the generator
    ## convertToImplicate: add candidate to the convertToImplicateList when it needs to be converted to an implicate before being passed to the generator
    ## solutions: add candidate to the solutions list when a candidate is a minimal conflict
    ### Function should return a tuple in the format (generatorAdditions, convertToImplicate, solutions)
    
    generatorAdditions = []
    convertToImplicate = []
    solutions = []
    
    ### YOUR CODE HERE ###
     
    return (generatorAdditions, convertToImplicate, solutions)
            

The following example problem simulates the candidate generator providing the candidate tester with 7 candidates. The target results of the candidate tester for this example are shown below.

In [ ]:
#Example problem: 7 Candidates given by the candidate generator to the candidate tester

candidate1 = frozenset([T1.get_assignment('thrust')])
candidate2 = frozenset([T1.get_assignment('nothrust')])
candidate4 = frozenset([R1.get_assignment('off')])
candidate1A = frozenset([T1.get_assignment('thrust'), R1.get_assignment('on')])
candidate1B = frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')])
candidate2A = frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on')])
candidate2B = frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')])


candidateList = [
    (candidate1, thruster_model[candidate1]),
    (candidate2, thruster_model[candidate2]),
    (candidate4, thruster_model[candidate4]),
    (candidate1A,thruster_model[candidate1][candidate1A]),
    (candidate1B,thruster_model[candidate1][candidate1B]),
    (candidate2A,thruster_model[candidate2][candidate2A]),
    (candidate2B,thruster_model[candidate2][candidate2B]),
]

#Test Prime Implicate Algorithm Output: 
(generatorAdditions, convertToImplicate, solutions) = prime_implicate_finder(sat, candidateList)
print("Generator Additions: ", generatorAdditions, "\n")
print("Convert To Implicate: ", convertToImplicate, "\n")
print("Solutions: ", solutions, "\n")

# Output should be: 
# Generator Additions:  [frozenset({(runthruster = off)}), frozenset({(thruster = nothrust), (runthruster = off)})] 
# Convert To Implicate:  [frozenset({(thruster = thrust)}), frozenset({(thruster = nothrust)}), frozenset({(thruster = thrust), (runthruster = on)}), frozenset({(thruster = nothrust), (runthruster = on)})] 
# Solutions:  [frozenset({(runthruster = off), (thruster = thrust)})] 
# Because 2 of the candidates are found to be valid, 1 is found to be unsatisfiable, and the remaining 4 candidates are determined to be satisfiable
# and are converted to implicates 

We now run a series of tests on larger examples to check the "prime_implicate_finder" function:

In [ ]:
## NB Grader Tests
check_prime_implicate_finder(prime_implicate_finder)
test_ok()

You're finished! Confirm that you've answered all 4 questions, and submit the homework.